## HLCM - Data Preprocessing & Model Estimation

Arezoo Besharati, UrbanSim, June 2018 


In [1]:
import os; os.chdir('../')
import numpy as np, pandas as pd 
# import matplotlib
# import matplotlib.pyplot as plt
from scipy.stats import skew

In [2]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'
    RED = '\033[91m'
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'

In [3]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca

/Users/arezoobzadeh/anaconda2/envs/ual-model/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load data

In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [5]:
orca.list_tables()

['parcels', 'buildings', 'units', 'households', 'persons', 'jobs']

### Generate Node variables

In [7]:
orca.run(['initialize_network'])

Running step 'initialize_network'
Time to execute step 'initialize_network': 16.18 s
Total time to execute iteration 1 with iteration value None: 16.18 s


In [8]:
orca.run(['network_aggregations'])

Running step 'network_aggregations'
Computing accessibility variables
Computing sum_income_3000
Removed 189769 rows because they contain missing values
Computing residential_units_500
Removed 4 rows because they contain missing values
Computing residential_units_1500
Removed 4 rows because they contain missing values
Computing population
Removed 189769 rows because they contain missing values
Computing poor
Removed 53114 rows because they contain missing values
Computing renters
Removed 102597 rows because they contain missing values
Computing ave_income_500
Removed 189769 rows because they contain missing values
Computing job_500
Computing job_1500
       sum_income_3000  residential_units_500  residential_units_1500  \
count     3.082600e+04           30826.000000            30826.000000   
mean      1.478451e+09               4.360019                6.175426   
std       1.927634e+09               2.391179                2.258851   
min       0.000000e+00               0.000000     

In [9]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

PARCELS
['development_type_id', 'land_value', 'acres', 'county_id', 'zone_id', 'proportion_undevelopable', 'tax_exempt_status', 'apn', 'parcel_id_local', 'geom_id', 'imputation_flag', 'x', 'y', 'shape_area', 'block_id', 'node_id']

BUILDINGS
['parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id', 'node_id']

UNITS
['Unnamed: 0', 'building_id', 'num_units', 'tenure', 'unit_num', 'unit_residential_price', 'unit_residential_rent', 'node_id']

HOUSEHOLDS
['household_id', 'serialno', 'persons', 'building_type', 'cars', 'income', 'race_of_head', 'hispanic_head', 'age_of_head', 'workers', 'state', 'county', 'tract', 'block group', 'children', 'tenure', 'recent_mover', 'block_group_id', 'single_family', 'un

In [10]:
#orca.list_broadcasts()

## Data Cleaning

 - Handling missing values
 - Checking the data types
 - Normalization: check for feature's distributions (the skewness)
 - Scalling: Check the units of data
 - Create dummy variables if needed


## 1. Buildings Table

### 1. 1. Missing values and data types

In [11]:
# Does data have missing values? 
bld = orca.get_table('buildings').to_frame()
print(color.RED + 'Data has missing values'+ color.END if bld.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ variables
print (color.BOLD +'Columns with missing values :\n'+ color.END + '{}'.format(bld.columns[bld.isna().any()].tolist()))
print ('')

# Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END+ '{}'.format(bld.dtypes[bld.dtypes == "object"]))


Data has missing values

Columns with missing values :
['redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'node_id']

String Features: 
redfin_home_type        object
costar_property_type    object
costar_rent             object
node_id                 object
dtype: object


costar_rent should be integer. 

In [12]:
# change the data type and update the column
bld.costar_rent = pd.to_numeric(bld.costar_rent, errors='coerce')

# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_bld = bld[['redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'node_id']].isnull().sum()
print ('')
print(color.BOLD + 'BUILDINGS' + color.END + '\nCount of missing points: \n{}'.format (missing_values_count_bld))
print('Proportion of missing points: \n{}'.format (missing_values_count_bld/ len (bld.costar_rent)))
print ('')



BUILDINGS
Count of missing points: 
redfin_sale_price        755143
redfin_sale_year         755143
redfin_home_type             78
costar_property_type         78
costar_rent             1835287
node_id                       4
dtype: int64
Proportion of missing points: 
redfin_sale_price       0.409658
redfin_sale_year        0.409658
redfin_home_type        0.000042
costar_property_type    0.000042
costar_rent             0.995625
node_id                 0.000002
dtype: float64



- 99% of costar rent is missing!! Don't use this feature
- 40 percent of redfin sale price and year is missing  

In [13]:
# drop the missing rows for "redfin_home_type" and "costar_property_type"       
bld.dropna(subset=['redfin_home_type', 'costar_property_type'],inplace = True)

## Update the whole table
orca.add_table('buildings', bld)

### 1. 2. Normalization

In [14]:
numeric_feats = bld.dtypes[bld.dtypes != "object"].index

skewed_feats = bld[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 1]
print(color.BOLD +'Skewed features are as follows'+ color.END + '\n{}'.format (skewed_feats))

Skewed features are as follows
development_type_id       5.016580
improvement_value       426.810509
residential_units        75.680060
residential_sqft        268.555659
sqft_per_unit             7.572658
non_residential_sqft    263.049391
building_sqft           182.761042
nonres_rent_per_sqft      5.122268
res_price_per_sqft       15.235614
stories                  15.571113
redfin_sale_price        62.174607
costar_rent               9.356972
building_type_id          4.316059
dtype: float64


In [15]:
skewed_feats = skewed_feats[['improvement_value', 'residential_sqft', 'sqft_per_unit','non_residential_sqft',
                            'building_sqft','nonres_rent_per_sqft','res_price_per_sqft','redfin_sale_price']]

In [16]:
skewed_feats = skewed_feats.index

bld[skewed_feats] = np.log1p(bld[skewed_feats])

# Update the building table 
orca.add_table('buildings', bld)

_____________________________________________________________________________________________________

## 2. Households Table

### 2. 1. Missing values and data types

In [17]:
# Does data have missing values? 
households= orca.get_table('households').to_frame()
print(color.RED + 'Data has missing values'+ color.END if households.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ variables
print ('Columns with missing values : {}'.format(households.columns[households.isna().any()].tolist()))
print ('')

# Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END+ '{}'.format(households.dtypes[households.dtypes == "object"]))

Data has missing values

Columns with missing values : ['workers', 'children', 'node_id']

String Features: 
hispanic_head    object
node_id          object
dtype: object


In [18]:
# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_hh = households[['workers', 'children', 'node_id']].isnull().sum()
print ('')
print(color.BOLD + 'HOUSEHOLDS' + color.END + '\nCount of missing points: \n{}'.format (missing_values_count_hh))
print('Proportion of missing points: \n{}'.format (missing_values_count_hh/ len (households.node_id)))


HOUSEHOLDS
Count of missing points: 
workers      581022
children    1835041
node_id      191984
dtype: int64
Proportion of missing points: 
workers     0.216825
children    0.684798
node_id     0.071644
dtype: float64


### Handling missing values

#### Option 1: Not using the features with missing values !

#### Option 2: Deleting/ignoring the missing values !!!

In [19]:
##I don't recommend this method at all. But in case one wants to do it 

#households.dropna(axis=0, how='any', inplace = True)

## Update the whole table
#orca.add_table('households', households)

#### Option 3:  Filling in the Value

In [20]:
# Impute with mean 
households.workers.fillna(households.workers.mean(), inplace = True)

# Update column
orca.get_table('households').update_col_from_series('workers', households.workers)

### 2. 2. Normalizing

In [21]:
# Only income needs to be normalized
households.income = np.log1p(households.income)

# Update column
orca.get_table('households').update_col_from_series('income', households.income)

/Users/arezoobzadeh/anaconda2/envs/ual-model/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


_____________________________________________________________________________________________________

## 3. Parcels Table

In [22]:
# Does data have missing values? 
parcels= orca.get_table('parcels').to_frame()
print(color.RED + 'Data has missing values'+ color.END if parcels.isnull().values.any() else 'Data doesnt have missing values')
print ('')

# What columns/ variables
print ('Columns with missing values : {}'.format(parcels.columns[parcels.isna().any()].tolist()))
print ('')


#Check features datatypes and see if there is any string feature that needs to be int/float
print(color.BOLD + 'String Features: \n'+ color.END + '{}'.format(parcels.dtypes[parcels.dtypes == "object"]))
print ('')

# Take a count of missing values. What proportion of the data is missing? 
missing_values_count_parcels = parcels[['x', 'y', 'block_id', 'node_id']].isnull().sum()
print(color.BOLD + 'PARCELS' + color.END + '\nCount of missing points: \n {}'.format(missing_values_count_parcels))
print('Proportion of missing points: \n{}'.format(missing_values_count_parcels/ len(parcels.node_id)))

Data has missing values

Columns with missing values : ['x', 'y', 'block_id', 'node_id']

String Features: 
apn                object
parcel_id_local    object
imputation_flag    object
block_id           object
node_id            object
dtype: object

PARCELS
Count of missing points: 
 x           4
y           4
block_id    5
node_id     4
dtype: int64
Proportion of missing points: 
x           0.000002
y           0.000002
block_id    0.000003
node_id     0.000002
dtype: float64


## Model Estimation

### Renters

In [23]:
df = orca.merge_tables(target='units', tables=['units','buildings','nodes'])

In [24]:
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['units','buildings','nodes']
m.choice_column = 'unit_id'
m.alt_sample_size = 50
m.chooser_filters = ['tenure == 2 & household_id <1000']

m.model_expression = 'res_price_per_sqft + population + ave_income_500 + job_500 + renters - 1'

m.name = 'hlcm_renter'
m.tags = ['arezoo', 'test']
m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:               
Model:         Multinomial Logit   Df Residuals:                   
Method:       Maximum Likelihood   Df Model:                       
Date:                              Pseudo R-squ.:                  
Time:                              Pseudo R-bar-squ.:              
AIC:                               Log-Likelihood:       -1,843.741
BIC:                               LL-Null:              -3,211.771
                        coef   std err         z     P>|z|   Conf. Int.
-----------------------------------------------------------------------
res_price_per_sqft   -0.9154     0.045   -20.392                       
population           -3.2970     0.208   -15.869                       
ave_income_500       -1.1700     0.046   -25.532                       
job_500               0.0431     0.010     4.374                       
renters               2.

In [28]:
m2 = LargeMultinomialLogitStep()
m2.choosers = ['households']
m2.alternatives = ['units','buildings','nodes']
m2.choice_column = 'unit_id'
m2.alt_sample_size = 50
m2.chooser_filters = ['tenure == 2 & household_id <1000']

m2.model_expression = 'res_price_per_sqft + population + ave_income_500:income + job_500 + renters - 1'

m2.name = 'hlcm_renter_2'
m2.tags = ['arezoo', 'test']
m2.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:               
Model:         Multinomial Logit   Df Residuals:                   
Method:       Maximum Likelihood   Df Model:                       
Date:                              Pseudo R-squ.:                  
Time:                              Pseudo R-bar-squ.:              
AIC:                               Log-Likelihood:       -1,849.557
BIC:                               LL-Null:              -3,211.771
                           coef   std err         z     P>|z|   Conf. Int.
--------------------------------------------------------------------------
res_price_per_sqft      -0.9049     0.041   -22.140                       
population              -3.3162     0.197   -16.815                       
ave_income_500:income   -0.1136     0.002   -50.915                       
job_500                  0.0415     0.010     4.219                       
renter

### Owners

In [26]:
m3 = LargeMultinomialLogitStep()
m3.choosers = ['households']
m3.alternatives = ['units','buildings','nodes']
m3.choice_column = 'unit_id'
m3.alt_sample_size = 50
m3.chooser_filters = ['tenure == 1 & household_id <1000']

m3.model_expression = ' population + ave_income_500 + job_500 + renters +year_built - 1'

m3.name = 'hlcm_owner'
m3.tags = ['arezoo', 'test','hlcm','owner']
m3.fit()


                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:               
Model:         Multinomial Logit   Df Residuals:                   
Method:       Maximum Likelihood   Df Model:                       
Date:                              Pseudo R-squ.:                  
Time:                              Pseudo R-bar-squ.:              
AIC:                               Log-Likelihood:         -546.277
BIC:                               LL-Null:                -700.252
                    coef   std err         z     P>|z|   Conf. Int.
-------------------------------------------------------------------
population       -3.1556     0.387    -8.145                       
ave_income_500    0.1490     0.348     0.428                       
job_500          -0.0213     0.020    -1.043                       
renters           2.1796     0.385     5.656                       
year_built       -0.0129     0.002    -7.228    

### Warnings:

Updating a column gives error if you want to convert obj (string) to integer


### To Do:
    
- create more node variables such as average household size 
- clean the redfin_home_type var from Buildings table
- check the skewness for node variables


### Cheat Sheet

In [27]:
# To add a new column to an existing orca table
#orca.add_column('name_of_the_table', 'new_column_name', new_column, cache=False, cache_scope='forever')